In [1]:
import tkinter as tk
import random

# Constants
WIDTH = 500
HEIGHT = 500
BALL_SIZE = 20
BALL_X_SPEED = 4
BALL_Y_SPEED = 4
PADDLE_WIDTH = 10
PADDLE_HEIGHT = 100#80
PADDLE_SPEED = 20#10
MAX_SCORE = 5

# Global variables
ball_x = BALL_X_SPEED
ball_y = BALL_Y_SPEED
paddle1_score = 0
paddle2_score = 0

# Create the window
root = tk.Tk()
root.title("Pong")

# Create the canvas
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="black")

# Create the ball
ball = canvas.create_oval(WIDTH/2-BALL_SIZE/2, HEIGHT/2-BALL_SIZE/2,
                          WIDTH/2+BALL_SIZE/2, HEIGHT/2+BALL_SIZE/2,
                          fill="white")

# Create the paddles
paddle1 = canvas.create_rectangle(0, HEIGHT/2-PADDLE_HEIGHT/2,
                                   PADDLE_WIDTH, HEIGHT/2+PADDLE_HEIGHT/2,
                                   fill="white")
paddle2 = canvas.create_rectangle(WIDTH-PADDLE_WIDTH, HEIGHT/2-PADDLE_HEIGHT/2,
                                   WIDTH, HEIGHT/2+PADDLE_HEIGHT/2,
                                   fill="white")

# Update the score
def update_score():
    global paddle1_score, paddle2_score,start

    canvas.delete("score")
    canvas.create_text(WIDTH/4, 50, text="Player 1: {}".format(paddle1_score),
                       font=("Arial", 24), fill="white", tag="score")
    canvas.create_text(3*WIDTH/4, 50, text="Player 2: {}".format(paddle2_score),
                       font=("Arial", 24), fill="white", tag="score")
    start=canvas.create_text(WIDTH/2, 100, text="Press \"r\" to start.",
                       font=("Arial", 24), fill="white", tag="score")

# Reset the ball
def reset_ball():
    global ball_x, ball_y

    # Move the ball to the center of the screen
    canvas.coords(ball, WIDTH/2-BALL_SIZE/2, HEIGHT/2-BALL_SIZE/2,
                  WIDTH/2+BALL_SIZE/2, HEIGHT/2+BALL_SIZE/2)

    # Give the ball a random starting direction
    ball_x = BALL_X_SPEED if random.choice([True, False]) else -BALL_X_SPEED
    ball_y = BALL_Y_SPEED if random.choice([True, False]) else -BALL_Y_SPEED

# Move the ball
def move_ball():
    global ball_x, ball_y, paddle1_score, paddle2_score
    # Move the ball
    canvas.move(ball, ball_x, ball_y)

    # Get the position of the ball
    ball_pos = canvas.coords(ball)

    # Get the position of the paddles
    paddle1_pos = canvas.coords(paddle1)
    paddle2_pos = canvas.coords(paddle2)

    # Bounce the ball off the top and bottom walls
    if ball_pos[1] <= 0 or ball_pos[3] >= HEIGHT:
        ball_y = -ball_y

    # Bounce the ball off the paddles
    if ball_pos[0] <= PADDLE_WIDTH and ball_pos[1] >= paddle1_pos[1] and ball_pos[3] <= paddle1_pos[3]:
        ball_x = BALL_X_SPEED
    elif ball_pos[2] >= WIDTH - PADDLE_WIDTH and ball_pos[1] >= paddle2_pos[1] and ball_pos[3] <= paddle2_pos[3]:
        ball_x = -BALL_X_SPEED
    else:
        # Check if the ball goes out of bounds
        if ball_pos[0] <= 0:
            paddle2_score += 1
            update_score()
            reset_ball()
        elif ball_pos[2] >= WIDTH:
            paddle1_score += 1
            update_score()
            reset_ball()

    # Schedule the next move of the ball
    after_id=canvas.after(20, move_ball)

# Check ball moving to avoid canvas.after increasing the speed of the ball
run_pressed=0
# Start the game when "r" key is pressed
def start_move_ball(event):
    global run_pressed
    # Check the pressed key, run the game if "r" key is pressed
    if event.keysym =="r" and run_pressed==0: 
        # Call move_ball to start the game
        move_ball()
        run_pressed +=1
        # Delete start text
        canvas.delete(start)
            
        
# Move the paddles
def move_paddle(event):
    global paddle1_score, paddle2_score

    # Get the position of the paddles
    paddle1_pos = canvas.coords(paddle1)
    paddle2_pos = canvas.coords(paddle2)

    # Move the paddles
    if event.keysym == "Up":
        if paddle2_pos[1] > 0:
            canvas.move(paddle2, 0, -PADDLE_SPEED)
    elif event.keysym == "Down":
        if paddle2_pos[3] < HEIGHT:
            canvas.move(paddle2, 0, PADDLE_SPEED)
    elif event.keysym == "w":
        if paddle1_pos[1] > 0:
            canvas.move(paddle1, 0, -PADDLE_SPEED)
    elif event.keysym == "s":
        if paddle1_pos[3] < HEIGHT:
            canvas.move(paddle1, 0, PADDLE_SPEED)

# Reset the game
def reset_game():
    global paddle1_score, paddle2_score
    ball_in_motion=False
    # Reset the scores
    paddle1_score = 0
    paddle2_score = 0

    # Update the score
    update_score()

    # Reset the ball
    reset_ball()
    
    run_pressed=0
    # Delete start text
    canvas.delete(start)

# Bind the reset_game function to the space bar
canvas.bind("<space>", lambda event: reset_game())

# Start the game
canvas.pack()
update_score()
canvas.focus_set()
canvas.bind("<Key>", move_paddle,add="+")
canvas.bind("<Key>", start_move_ball, add="+")

# Show the window
root.mainloop()